In [1]:
# === 1. Setup & Imports
from QuantumRingsLib import QuantumRingsProvider
from quantumrings.toolkit.qiskit import QrBackendV2  
from qiskit import QuantumCircuit
from qiskit.visualization import plot_bloch_multivector
from qiskit.quantum_info import Statevector
from qiskit.visualization.bloch import Bloch
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from qiskit.circuit.library import RVGate, StatePreparation
from io import BytesIO
from IPython.display import HTML
from PIL import Image
from matplotlib.animation import FuncAnimation

# Load saved account and select noise-free backend
provider = QuantumRingsProvider()
backend = provider.get_backend("scarlet_quantum_rings")

In [2]:
def gate(initial_state, rotation_axis, t):
    
    qc = QuantumCircuit(1)

    stateprep = StatePreparation(initial_state)
    qc.append(stateprep, [0])

    # rescale by time
    vector = 2*np.pi*t * rotation_axis

    gate = RVGate(vector[0],vector[1],vector[2])

    qc.append(gate, [0])
    
    sv = Statevector.from_instruction(qc)

    return sv

def gate_animation(rotation_axis, initial_state=[1,0], rotation_angle=np.pi, 
                   num_frames=21, interval=100, repeat=False):

    # normalize
    rotation_axis = np.array(rotation_axis)
    length = np.sqrt(np.sum(rotation_axis**2))
    normalized_axis = rotation_axis/length

    # Generate all Bloch vector images
    frames = []
    for i in range(0, num_frames+1):
        t = i / num_frames
        bloch = gate(initial_state, normalized_axis, t * rotation_angle/(2*np.pi))

        alpha, beta = bloch.data  # amplitudes for |0> and |1>

        # Compute Bloch vector
        x = 2 * np.real(np.conj(alpha) * beta)
        y = 2 * np.imag(np.conj(alpha) * beta)
        z = np.abs(alpha)**2 - np.abs(beta)**2
        
        bloch_vector = [x, y, z]
        
        fig = plot_bloch_multivector(bloch)
        b = Bloch()
        b.add_vectors(normalized_axis)  # add the rotation axis vector
        b.add_annotation(rotation_axis, "Rotation Axis", fontsize=16)
        b.add_vectors(bloch_vector)  # add the state vector
        
        buf = BytesIO()
        b.save(name=buf)
        buf.seek(0)
        img = Image.open(buf)
        frames.append(img.copy())
        plt.close(fig)

    # Convert PIL Images to NumPy arrays
    frame_arrays = [np.asarray(img) for img in frames]
    
    # Set up figure
    fig, ax = plt.subplots()
    im = ax.imshow(frame_arrays[0])
    ax.axis('off')  # Optional: hide axes
    
    # Update function for animation
    def update(frame_index):
        im.set_array(frame_arrays[frame_index])
        return [im]
    
    # Create the animation
    ani = FuncAnimation(fig, update, frames=len(frame_arrays), interval=interval, repeat=repeat)
    
    # Display inline
    plt.close(fig)
    return HTML(ani.to_jshtml())
    

In [6]:
axis = [1,1,0]
rotation_angle = 2*np.pi
initial_state = [0,1]

gate_animation(axis, initial_state=initial_state, rotation_angle=rotation_angle)